In [ ]:
#%%timeit -n1 -r1
# Импортируем наш модуль
from movielens_analysis import Ratings, Tags, Movies, Links

# Указываем путь к файлам 
ratings_file = '../datasets/ml-latest-small/ratings.csv'
tags_file = '../datasets/ml-latest-small/tags.csv'
movies_file = '../datasets/ml-latest-small/movies.csv'
links_file = '../datasets/ml-latest-small/links.csv'

#Пути к сломанным файлам
# ratings_file = '../datasets/b_file/ratings.csv'
# tags_file = '../datasets/b_file/tags.csv'
# movies_file = '../datasets/b_file/movies.csv'
# links_file = '../datasets/b_file/links.csv'

# Создаем объекты классов
movies = Movies(movies_file)
movies_list = movies.get_movies()
movie_ids = set(int(m["movieId"]) for m in movies_list)
ratings_obj = Ratings(ratings_file, movies_file, movie_ids)
movies_analytics = Ratings.Movies(ratings_obj, movies_list)
movies_analytics2 = Ratings.Users(ratings_obj, movies_list)
tags = Tags(tags_file, movie_ids)
links = Links(links_file)
imdb_ids = links.read_csv_column(links_file, 'imdbId', movie_ids)[:10]

# Анализ набора данных MovieLens: Как меняются вкусы зрителей в зависимости от года выхода фильма?

**Дата создания отчета:** 2025-07-12

**Авторы**

- **mashalee** (teamlead)
- **iridescm**
- **korisund**

В этом исследовании мы исследуем набор данных MovieLens и постараемся понять, как менялись предпочтения зрителей в зависимости от года выхода фильма. Какие жанры были наиболее любимыми в 90-х? А какие — в 2000-х? Менялся ли средний рейтинг со временем? Какие фильмы выделялись особенно? Как пользовательчкие теги могут быть связаны со средними рейтингами этих фильмов?

## Сбор данных

Рассмотрим все классы и методы, которые нам помогут собрать данные для анализа.

## 1. Класс Movies

Соберем основную информацию о фильмах с помощью класса `Movies`. Данный класс предназначен для хранения и организации информации о фильмах.  
Данные которые берутся из файла:

- `movieId` — уникальный идентификатор фильма;
- `title` — название фильма;
- `genres` — список жанров, к которым относится фильм.

Данная структура обеспечивает удобный доступ к информации о фильмах и позволяет эффективно работать с жанровыми категориями.  
Для анализа нам необходимо расширить класс дополнительными атрибутами, такими как год выпуска, рейтинг и возможно др.

### 1.1 Метод dist_by_release(self)
Для полной картины нам необходимо понимать в какие года выпускались фильмы. С помощью данного метода мы создадим словарь, ключами которого являются года, а значениями количество фильмом выпущенных в этом году.

Теперь мы легко сможем проанализировать фильмы каких годов содержатся в наших данных. Данные отсортированы по убыванию по количеству фильмов.

In [9]:
%%timeit -r1 -n1
display(movies.dist_by_release())

OrderedDict([('1995', 224),
             ('1994', 184),
             ('1996', 181),
             ('1993', 101),
             ('1992', 23),
             ('1990', 15),
             ('1991', 15),
             ('1989', 14),
             ('1986', 9),
             ('1982', 8),
             ('1940', 8),
             ('1957', 8),
             ('1987', 8),
             ('1980', 8),
             ('1981', 7),
             ('1988', 7),
             ('1979', 7),
             ('1955', 6),
             ('1959', 6),
             ('1968', 6),
             ('1997', 6),
             ('1939', 6),
             ('1985', 6),
             ('1967', 5),
             ('1965', 5),
             ('1951', 5),
             ('1958', 5),
             ('1944', 5),
             ('1941', 5),
             ('1975', 5),
             ('1971', 5),
             ('1984', 5),
             ('1964', 4),
             ('1973', 4),
             ('1954', 4),
             ('1934', 4),
             ('1960', 4),
             ('1963', 4),


6.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Мы видим, что в 1995 году было снято больше всего фильмов. Значит этот год представляет для нас самый большой интерес. 

### 1.2 Метод dist_by_genres(self)

Для анализа необходимо понимать к каким жанрам принадлежат фильмы. 

Метода `dist_by_genres(self)` позволяет получить распределение фильмов по жанрам, представленных в наборе данных. Он возвращает список всех жанров и количество фильмов, относящихся к каждому из них.

In [10]:
%%timeit -n1 -r1
display(movies.dist_by_genres())

{'Drama': 507,
 'Comedy': 365,
 'Romance': 208,
 'Thriller': 179,
 'Action': 158,
 'Adventure': 126,
 'Crime': 122,
 'Children': 100,
 'Fantasy': 69,
 'Sci-Fi': 69,
 'Mystery': 58,
 'Musical': 53,
 'Horror': 51,
 'War': 48,
 'Animation': 37,
 'Documentary': 25,
 'Western': 23,
 'Film-Noir': 18,
 'IMAX': 3}

1.76 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Драма - самый популярный жанр у режиссеров. Или это не совсем так? По одному ли жанру имеет каждый фильм? 

### 1.3 Метод most_genres(n)

После анализа годов выпуска и жанров, содержащихся в данных, надо понимать какое самое максимальное количество жанров указано в одном фильме.

Метод `most_genres(n)` выводит созданный словарь с фильмами, которые имеют больше всего жанров. 
Ключ- название фильма. Значение- количество жанров в нём.

Выведем Топ-10 фильмов.

In [11]:
%%timeit -n1 -r1
display(dict(movies.most_genres(10)))

{'Strange Days (1995)': 6,
 'Lion King, The (1994)': 6,
 'Getaway, The (1994)': 6,
 'Super Mario Bros. (1993)': 6,
 'Beauty and the Beast (1991)': 6,
 'All Dogs Go to Heaven 2 (1996)': 6,
 'Space Jam (1996)': 6,
 'Aladdin and the King of Thieves (1996)': 6,
 'Toy Story (1995)': 5,
 'Money Train (1995)': 5}

2.31 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Мы видим, что есть фильмы, у которых указано очень много жанров.
А какое же сочетание жанров встречается чаще всего?

### 1.4* common_genre_combinations(self, n=10)

Чтоб исследовать вопрос, воспользуемся методом 'common_genre_combinations'. Метод выводит словарь, в котором указан жанр или сочетание жанров и число, с частотой которого встречатеся фильм.

In [12]:
%%timeit -r1 -n1
display(movies.common_genre_combinations(10))

{'Drama': 142,
 'Comedy': 94,
 'Comedy|Drama': 54,
 'Drama|Romance': 51,
 'Comedy|Romance': 43,
 'Comedy|Drama|Romance': 28,
 'Documentary': 23,
 'Drama|Thriller': 20,
 'Crime|Drama': 19,
 'Drama|War': 17}

2.49 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 1.5* movies_by_year(self, year)

Давайте теперь посмотрим какие именно фильмы были созданны в 1995 году и какому жанру они относятся. Поймем какой жанр был предпочтительней у режиссеров, выведим 10 фильмов, чтоб в этом убедится.


In [13]:
%%timeit -n1 -r1
display(movies.movies_by_year(1995)[:10])

total movies: 224, most popularity genre: ('Drama', 107)


[{'Money Train (1995)': ['Action', 'Comedy', 'Crime', 'Drama', 'Thriller']},
 {'Bad Boys (1995)': ['Action', 'Comedy', 'Crime', 'Drama', 'Thriller']},
 {'Strange Days (1995)': ['Action',
   'Crime',
   'Drama',
   'Mystery',
   'Sci-Fi',
   'Thriller']},
 {'Waiting to Exhale (1995)': ['Comedy', 'Drama', 'Romance']},
 {'American President, The (1995)': ['Comedy', 'Drama', 'Romance']},
 {'Mighty Aphrodite (1995)': ['Comedy', 'Drama', 'Romance']},
 {'Something to Talk About (1995)': ['Comedy', 'Drama', 'Romance']},
 {'Don Juan DeMarco (1995)': ['Comedy', 'Drama', 'Romance']},
 {'To Die For (1995)': ['Comedy', 'Drama', 'Thriller']},
 {'Doom Generation, The (1995)': ['Comedy', 'Crime', 'Drama']}]

3.43 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### Вывод:
Данные показали нам, что фильмы с жанром "драма" снимают чаще всего. На сколько этот жанр популярен на самом деле попробуем узнать с помощью следующего класса.


## 2. Класс Ratings

Класс `Ratings` предназначен для хранения и организации информации о том, какие оценки поставили пользователи фильму.  
Данные которые берутся из файла:

- `userId` — уникальный идентификатор пользователя;
- `movieId` — уникальный идентификатор фильма;
- `rating` — оценка, которую поставил пользователь;
- `timestamp` — дата и время выбора выставления оценки.

Данная структура обеспечивает удобный доступ к информации о том, какие оценки ставили фильму. Благодаря этому мы можем узнать какие фильмы высоко оцениваются зрителями.

Давайте узнаем в какие годы, сколько оценок было выставленно.

### 2.1 Метод dist_by_year(self)

Метод возвращает словарь, где годам соответствует количество выставленных оценок.

In [14]:
%%timeit -n1 -r1
display(movies_analytics.dist_by_year())

{1996: 453,
 1998: 36,
 1999: 47,
 2000: 127,
 2001: 21,
 2003: 11,
 2005: 31,
 2009: 12,
 2011: 67,
 2012: 2,
 2013: 52,
 2015: 2,
 2016: 111,
 2017: 26,
 2018: 2}

1.98 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Мы видим, что фильмы в 1996 года оценивались больше всего. Интересно какие это были оценки?

### 2.2 dist_by_rating(self)

А какие оценки чаще всего ставят пользователи? Сейчас узнаем!

Метод возвращает словарь оценок и их количества.

In [15]:
%%timeit -n1 -r1
display(movies_analytics.dist_by_rating())

{0.5: 11,
 1.0: 26,
 1.5: 2,
 2.0: 47,
 2.5: 13,
 3.0: 287,
 3.5: 41,
 4.0: 303,
 4.5: 59,
 5.0: 211}

1.81 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Большинство пользователей ставят высокие оценки (от 3.0 до 5.0). Наиболее частая оценка — 4.0, за ней следует 5.0 и 3.0.
Низкие оценки встречаются редко, что говорит о высокой лояльности или качестве фильмов. Распределение смещено в сторону положительных оценок

### 2.3 top_by_num_of_ratings(self, n)

А какие же фильмы самые хайповые? Кому больше всего поставили оценки? Сейчас посмотрим!

Этот метод возвращает словарь, где названиям фильма соответствуют количество оценок.

In [16]:
%%timeit -n1 -r1
display(movies_analytics.top_by_num_of_ratings(10))

{'Pulp Fiction (1994)': 11,
 'Forrest Gump (1994)': 11,
 'Seven (a.k.a. Se7en) (1995)': 10,
 "Schindler's List (1993)": 10,
 'Silence of the Lambs, The (1991)': 10,
 'Shawshank Redemption, The (1994)': 10,
 'Usual Suspects, The (1995)': 8,
 'Braveheart (1995)': 8,
 'Jurassic Park (1993)': 8,
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)': 8}

1.82 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Интересно, что фильмы 90-х годов оценивались чаще. 

### 2.4 top_by_ratings(self, n, metric='average')

Много оценок - это хорошо. Но не понятно хорошие они или плохие. Сейчас выясним.

Метод возвращает словарь, который названию фильма ставит в соответствие либо среднее значение, либо медиану.

In [17]:
%%timeit -n1 -r1
display(movies_analytics.top_by_ratings(10, 'average'))

{'Ghost and the Darkness, The (1996)': 5.0,
 'Fantasia (1940)': 5.0,
 'First Knight (1995)': 5.0,
 "Monty Python's Life of Brian (1979)": 4.83,
 'Clockwork Orange, A (1971)': 4.83,
 'Alice in Wonderland (1951)': 4.75,
 'Blues Brothers, The (1980)': 4.75,
 'Back to the Future (1985)': 4.75,
 'Godfather: Part II, The (1974)': 4.75,
 'Tommy Boy (1995)': 4.67}

2.24 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [18]:
%%timeit -n1 -r1
display(movies_analytics.top_by_ratings(10, 'median'))

{'Star Wars: Episode IV - A New Hope (1977)': 5.0,
 'Tommy Boy (1995)': 5.0,
 'Fugitive, The (1993)': 5.0,
 'Wizard of Oz, The (1939)': 5.0,
 'Citizen Kane (1941)': 5.0,
 'Ghost and the Darkness, The (1996)': 5.0,
 "Monty Python's Life of Brian (1979)": 5.0,
 'Star Wars: Episode V - The Empire Strikes Back (1980)': 5.0,
 'Clockwork Orange, A (1971)': 5.0,
 'Terminator, The (1984)': 5.0}

2.22 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Из-за того, что мы берём только ограниченное количество данных, у нас много фильмов, где выставлена одна оценка. И если это "5", то фильм попадёт в топ нашего рейтинга.

### 2.5 top_controversial(self, n)

А большой ли разброс между оценками. Этот метод нам поможет выяснить. Топ-n самых спорных фильмов, то есть те, у которых пользователи больше всего расходились в оценках.

Метод возвращает словарь, который выбирает первые n фильмов с самой большой дисперсией.

In [19]:
%%timeit -r1 -n1
display(movies_analytics.top_controversial(10))

{'My Fair Lady (1964)': 10.12,
 "City Slickers II: The Legend of Curly's Gold (1994)": 8.0,
 'Courage Under Fire (1996)': 6.12,
 'Doors, The (1991)': 6.12,
 'Deer Hunter, The (1978)': 6.12,
 'Dazed and Confused (1993)': 4.5,
 'Brazil (1985)': 4.5,
 'Casper (1995)': 4.5,
 'Flintstones, The (1994)': 4.5,
 'Circle of Friends (1995)': 4.33}

2.64 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 2.6* average_genre_rating_by_year(self, genre_filter=None, release_year=None)

Хотелось бы узнать, как менялось восприятие разных жанров по годам. Рассмотрим жанр «Драма», поскольку фильмы этого жанра встречаются чаще всего.

Данный метод рассчитывает средние рейтинги фильмов заданного жанра с разбивкой по году выпуска фильма и показывает, в каком году была поставилена оценка, а также их количество в каждом году.

In [20]:
%%timeit -r1 -n1
display(movies_analytics.average_genre_rating_by_year(genre_filter=None, release_year=1995))

{1996: {'count': 153, 'average_rating': 3.48},
 1998: {'count': 11, 'average_rating': 3.73},
 1999: {'count': 6, 'average_rating': 2.5},
 2000: {'count': 18, 'average_rating': 4.06},
 2001: {'count': 3, 'average_rating': 3.67},
 2003: {'count': 3, 'average_rating': 3.0},
 2005: {'count': 4, 'average_rating': 4.38},
 2009: {'count': 3, 'average_rating': 4.67},
 2011: {'count': 11, 'average_rating': 2.59},
 2013: {'count': 4, 'average_rating': 3.38},
 2015: {'count': 1, 'average_rating': 4.0},
 2016: {'count': 16, 'average_rating': 3.88},
 2017: {'count': 2, 'average_rating': 3.0}}

3.81 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [21]:
%%timeit -r1 -n1
display(movies_analytics.average_genre_rating_by_year(genre_filter="Drama", release_year=None))

{1996: {'count': 208, 'average_rating': 3.66},
 1998: {'count': 11, 'average_rating': 4.27},
 1999: {'count': 22, 'average_rating': 3.91},
 2000: {'count': 41, 'average_rating': 4.12},
 2001: {'count': 9, 'average_rating': 2.89},
 2003: {'count': 6, 'average_rating': 3.67},
 2005: {'count': 9, 'average_rating': 3.17},
 2009: {'count': 6, 'average_rating': 4.5},
 2011: {'count': 28, 'average_rating': 3.25},
 2012: {'count': 2, 'average_rating': 4.25},
 2013: {'count': 23, 'average_rating': 3.8},
 2015: {'count': 1, 'average_rating': 3.0},
 2016: {'count': 42, 'average_rating': 4.02},
 2017: {'count': 10, 'average_rating': 4.15},
 2018: {'count': 1, 'average_rating': 4.5}}

5.15 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [22]:
%%timeit -r1 -n1
display(movies_analytics.average_genre_rating_by_year(genre_filter="Drama", release_year=1994))

{1996: {'count': 64, 'average_rating': 3.72},
 1998: {'count': 3, 'average_rating': 4.67},
 1999: {'count': 4, 'average_rating': 3.0},
 2000: {'count': 6, 'average_rating': 3.83},
 2001: {'count': 4, 'average_rating': 3.0},
 2003: {'count': 1, 'average_rating': 3.0},
 2005: {'count': 2, 'average_rating': 4.0},
 2009: {'count': 2, 'average_rating': 3.75},
 2011: {'count': 4, 'average_rating': 4.62},
 2013: {'count': 5, 'average_rating': 3.7},
 2015: {'count': 1, 'average_rating': 3.0},
 2016: {'count': 9, 'average_rating': 3.78},
 2017: {'count': 5, 'average_rating': 4.0}}

4.06 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


В 1996 году пользователи поставили больше всего оценок — 64. В следующие годы количество оценок стало намного меньше.
Это значит, что фильмы этого жанра в 90-х годах были популярнее и получали больше оценок, а в последние годы — меньше.

### 2.7 dist_by_num_of_ratings(self, min_ratings=None)

Теперь пришло взглянуть не только на фильмы, но и на пользователей, которые ставят оценки.

Метод возвращает словарь, в котором каждой оценке соответствует количество пользователей, поставивших её.

In [23]:
%%timeit -r1 -n1
display(movies_analytics2.dist_by_num_of_ratings())

{0.5: 2,
 1.0: 8,
 1.5: 1,
 2.0: 13,
 2.5: 4,
 3.0: 17,
 3.5: 8,
 4.0: 17,
 4.5: 5,
 5.0: 16}

1.84 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 2.8 dist_by_user_rating(self, metric="average")

А какие в среднем оценки ставил каждый пользователь? А какая медиана оценок? Давайте выясним.

Метод возвращает словарь, где средней и медиальной оценке соответствует сколько пользователей ставили такую оценку.

In [24]:
%%timeit -r1 -n1
display(movies_analytics2.dist_by_user_rating('average'))

{1.1: 1,
 2.8: 2,
 3.4: 2,
 3.5: 3,
 3.6: 2,
 3.7: 2,
 3.8: 2,
 3.9: 2,
 4.2: 1,
 4.3: 1,
 4.4: 1}

1.71 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [25]:
%%timeit -r1 -n1
display(movies_analytics2.dist_by_user_rating('median'))

{0.5: 1, 3.0: 5, 3.2: 1, 3.5: 1, 3.8: 1, 4.0: 7, 4.2: 1, 4.5: 1, 5.0: 1}

1.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 2.9 top_controversial(self, n)

Для того, чтобы узнать на сколько большой разрос оценок, который ставит пользователь, воспользуемся методом `top_controversial(self, n)`.

Метод возвращает топ-N пользователей с наибольшей дисперсией их оценок (т.е. кто оценивает наиболее непоследовательно)

In [26]:
%%timeit -r1 -n1
display(movies_analytics2.top_controversial(10))

{13: 2.8,
 3: 2.23,
 15: 2.04,
 10: 1.78,
 4: 1.77,
 7: 1.1,
 14: 1.05,
 9: 1.02,
 5: 0.98,
 8: 0.95}

2.04 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 2.10* genre_rating_trend_by_year(self, genre_filter: str = "Drama")

Как менялось отношение к жанру "Drama" или "Thriller" по годам, когда люди ставили оценки?

Для анализа можно использовать метод, который возвращает словарь, где ключ — год выставления оценки, а значение — другой словарь с тремя данными: средней оценкой, количеством оценок и количеством пользователей, которые оценивали фильмы этого жанра в этом году.

In [27]:
%%timeit -r1 -n1
display(movies_analytics2.genre_rating_trend_by_year("Thriller"))

{1996: {'Средний рейтинг': 3.57, 'оценок': 106, 'пользователей': 4},
 1998: {'Средний рейтинг': 3.84, 'оценок': 19, 'пользователей': 1},
 1999: {'Средний рейтинг': 3.22, 'оценок': 9, 'пользователей': 1},
 2000: {'Средний рейтинг': 4.11, 'оценок': 28, 'пользователей': 3},
 2001: {'Средний рейтинг': 3.67, 'оценок': 3, 'пользователей': 2},
 2003: {'Средний рейтинг': 2.5, 'оценок': 2, 'пользователей': 1},
 2005: {'Средний рейтинг': 4.0, 'оценок': 10, 'пользователей': 1},
 2009: {'Средний рейтинг': 3.5, 'оценок': 1, 'пользователей': 1},
 2011: {'Средний рейтинг': 4.0, 'оценок': 18, 'пользователей': 3},
 2012: {'Средний рейтинг': 4.0, 'оценок': 1, 'пользователей': 1},
 2013: {'Средний рейтинг': 3.94, 'оценок': 16, 'пользователей': 1},
 2016: {'Средний рейтинг': 3.78, 'оценок': 34, 'пользователей': 2},
 2017: {'Средний рейтинг': 3.71, 'оценок': 7, 'пользователей': 2}}

2.29 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [28]:
%%timeit -r1 -n1
display(movies_analytics2.genre_rating_trend_by_year("Drama"))

{1996: {'Средний рейтинг': 3.66, 'оценок': 208, 'пользователей': 4},
 1998: {'Средний рейтинг': 4.27, 'оценок': 11, 'пользователей': 1},
 1999: {'Средний рейтинг': 3.91, 'оценок': 22, 'пользователей': 1},
 2000: {'Средний рейтинг': 4.12, 'оценок': 41, 'пользователей': 2},
 2001: {'Средний рейтинг': 2.89, 'оценок': 9, 'пользователей': 2},
 2003: {'Средний рейтинг': 3.67, 'оценок': 6, 'пользователей': 1},
 2005: {'Средний рейтинг': 3.17, 'оценок': 9, 'пользователей': 1},
 2009: {'Средний рейтинг': 4.5, 'оценок': 6, 'пользователей': 1},
 2011: {'Средний рейтинг': 3.25, 'оценок': 28, 'пользователей': 2},
 2012: {'Средний рейтинг': 4.25, 'оценок': 2, 'пользователей': 1},
 2013: {'Средний рейтинг': 3.8, 'оценок': 23, 'пользователей': 1},
 2015: {'Средний рейтинг': 3.0, 'оценок': 1, 'пользователей': 1},
 2016: {'Средний рейтинг': 4.02, 'оценок': 42, 'пользователей': 2},
 2017: {'Средний рейтинг': 4.15, 'оценок': 10, 'пользователей': 2},
 2018: {'Средний рейтинг': 4.5, 'оценок': 1, 'пользовате

3.47 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## 3. Класс Tags

Класс `Tags` предназначен для хранения и организации информации о том, какие тэги поставили пользователи фильму.  
Данные которые берутся из файла:

- `userId` — уникальный идентификатор пользователя;
- `movieId` — уникальный идентификатор фильма;
- `tag` — тэг, который поставил пользователь;
- `timestamp` — дата и время выбора тэга.

Данная структура обеспечивает удобный доступ к информации о том, какие тэги соответствуют фильму. Благодаря этому посетители сайта могут лучше ориентироваться в огромном множестве фильмов.

### 3.1 most_words(self, n)

Некоторые пользователи в тэгах могут написать огромный текст. Попробуем найти самые длинные или составные теги, которые могут отражать более сложные или специфические характеристики фильмов.

Метод возвращает словарь, который каждому тэгу фильма указывает количество слов. Возвращает n тегов с максимальным количеством слов в названии тега. Чтобы найти самые "длинные" или многословные теги. `{тег: кол-во слов в теге}`

In [29]:
%%timeit -r1 -n1
display(tags.most_words(10))

{'villain nonexistent or not needed for good story': 8,
 'It was melodramatic and kind of dumb': 7,
 'Oscar (Best Effects - Visual Effects)': 6,
 'Oscar (Best Music - Original Score)': 6,
 'stop looking at me swan': 5,
 'based on a true story': 5,
 'based on a TV show': 5,
 'GIVE ME BACK MY SON!': 5,
 'stupid is as stupid does': 5,
 'rich guy - poor girl': 5}

2.11 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 3.2* tag_statistics(self, movies_obj)

Если предыдущий метод анализировал сами теги, то этот поможет понять, какие фильмы более активно тегированы, то есть у каких фильмов больше всего тегов. Это может помочь выявить популярные или «обсуждаемые» фильмы. 
Возвращает статистику по фильмам: сколько тегов у каждого фильма.
Результат — словарь, где ключ — название фильма (или ID, если название не передано), а значение — количество тегов у этого фильма.`{movie_title_or_id: кол-во тегов для фильма}`

In [30]:
%%timeit -r1 -n1
display(dict(list(tags.tag_statistics(movies).items())[:15]))

{'Pulp Fiction (1994)': 181,
 '2001: A Space Odyssey (1968)': 41,
 'Léon: The Professional (a.k.a. The Professional) (Léon) (1994)': 35,
 'Star Wars: Episode IV - A New Hope (1977)': 26,
 'Blade Runner (1982)': 13,
 'Reservoir Dogs (1992)': 11,
 'Braveheart (1995)': 10,
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)': 10,
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)': 10,
 'Star Wars: Episode V - The Empire Strikes Back (1980)': 10,
 'Aliens (1986)': 9,
 'Psycho (1960)': 9,
 'Shining, The (1980)': 9,
 'Forrest Gump (1994)': 9,
 '12 Angry Men (1957)': 9}

2.05 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 3.3* get_tags_for_movie(self, title, movies_obj)

О! Психо мой любимый фильм. Давайте посмотрим что же писали пользователи?

Метод принимает на вход название фильма и печатает все тэги по этому фильму.

In [31]:
%%timeit -r1 -n1
display(tags.get_tags_for_movie('Psycho (1960)', movies))

['Alfred Hitchcock',
 'psychology',
 'suspenseful',
 'tense',
 'Norman Bates',
 'Alfred Hitchcock',
 'black and white',
 'imdb top 250',
 'remade']

1.72 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 3.4 longest(self, n)

Слов то может быть и мало в тэги, но они могут быть большими! Либо наоборот. Сейчас выясним.

Метод возвращает список самых длиных тэгов по количеству символов.

In [32]:
%%timeit -r1 -n1
display(tags.longest(10))

['villain nonexistent or not needed for good story',
 'r:disturbing violent content including rape',
 'Oscar (Best Effects - Visual Effects)',
 'It was melodramatic and kind of dumb',
 'r:sustained strong stylized violence',
 'Oscar (Best Music - Original Score)',
 'start of a beautiful friendship',
 'Oscar (Best Cinematography)',
 'r:disturbing violent images',
 'r:strong bloody violence']

1.51 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 3.5 most_words_and_longest(self, n)

Мы посмотрели на самые длиные тэги, на тэги с большим количеством слов. Ну что ж пора выяснить, а если ли среди них совпадающие тэги.

Этот метод выводит список совпадений из двух предыдущих методов. Возвращает теги, которые одновременно входят в топ n по количеству слов и по длине строки. Это пересечение двух списков: most_words(n) — топ по количеству слов; longest(n) — топ по длине.

In [33]:
%%timeit -r1 -n1
display(tags.most_words_and_longest(10))

['villain nonexistent or not needed for good story',
 'It was melodramatic and kind of dumb',
 'Oscar (Best Effects - Visual Effects)',
 'Oscar (Best Music - Original Score)']

2.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 3.6 most_popular(self, n)

Люди не славятся оригинальностью, в отличии от ИИ, который написал этот отчёт. Шутка! Хотя вряд ли её кто-то прочтёт. Давай-то взглянет на то, сколько одинаковых тэгов у нас.

Метод возвращает словарь n самых часто встречающихся тегов и их количество

In [34]:
%%timeit -r1 -n1
display(tags.most_popular(10))

{'In Netflix queue': 17,
 'Disney': 16,
 'space': 9,
 'dark comedy': 8,
 'sci-fi': 8,
 'Mafia': 7,
 'music': 7,
 'time travel': 7,
 'aliens': 7,
 'atmospheric': 7}

2.02 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 3.7 tags_with(self, word)

А что будем делать, если захотим поискать слово в тэге?

Метод возвращает список тэгов, в которых встречается подстрока. Возвращает все уникальные теги, содержащие заданное слово (без учёта регистра).

In [35]:
%%timeit -n1 -r1
display(tags.tags_with('music'))

['Music',
 'Oscar (Best Music - Original Score)',
 'drugs & music',
 'good music',
 'music']

1.56 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 3.6* top_movies_by_tag(self, tag_name, ratings_obj, movies_obj, n=10)

А как же узнать какие фильмы с конкретным тегом больше всего понравились зрителям. Нам надо найти Топ-n фильмов по тегу и посмотреть какую среднюю оценку они получили.

Этот метод помогает найти лучшие фильмы, которым был присвоен конкретный тег. Возвращает топ-n фильмов с данным тегом по средней оценке. `dict: Словарь формата {название фильма: средняя оценка}, отсортированный по убыванию оценки`. 

Сноска: Из-за того, что мы берем только 1000 строк из файла, то не все рейтинги и названия попали, поэтому список короче, чем найдено тегов.

In [36]:
%%timeit -r1 -n1
display(tags.top_movies_by_tag('In Netflix queue', ratings_obj, movies, 10))

{'Crumb (1994)': 5.0,
 'Once Were Warriors (1994)': 5.0,
 'Eat Drink Man Woman (Yin shi nan nu) (1994)': 4.5,
 'Lone Star (1996)': 4.0,
 'When We Were Kings (1996)': 4.0,
 'Secret Garden, The (1993)': 3.5,
 'My Family (1995)': 3.0}

2.31 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## 4. Класс Links

Класс `Links` предназначен для хранения и организации информации о фильмах, которые мы парсим с сайте и нескольких файлов.

Эти данные мы обрабатываем:

- `movieId` — уникальный идентификатор фильма;
- `imdbId` — уникальный идентификатор фильма на сайте imdb;
- `tmdbId` — уникальный идентификатор фильма на сайте tmdb;
- `title` — название фильма;
- `director` — директор фильма;
- `writers` — список сценаристов;
- `stars` — список звёзд-актёров;
- `budget` — бюджет фильма;
- `worldwide` — мировые сборы фильма ;
- `runtime` — продолжительность фильма в минутах;
- `raiting` — рейтинг фильма.

Данная структура обеспечивает удобный доступ к информации о фильме.

### 4.1 get_imdb(self, list_of_movies, list_of_fields)

Мы спарсили данные с сайта imdb и теперь выводим список фильмов с соответсвующими полями.

Данный метод формирует: список списков

In [37]:
%%timeit -r 1 -n1
a = links.get_imdb(imdb_ids, ['Director', 'Budget', 'Gross worldwide', 'Runtime', 'Title'])
display(a)

[['0114885',
  'Forest Whitaker',
  '$16,000,000 (estimated)',
  '$81,452,156',
  '2 hours 4 minutes',
  'В ожидании выдоха'],
 ['0114709',
  'John Lasseter',
  '$30,000,000 (estimated)',
  '$394,436,586',
  '1 hour 21 minutes',
  'История игрушек'],
 ['0114576',
  'Peter Hyams',
  '$35,000,000 (estimated)',
  '$64,350,171',
  '1 hour 51 minutes',
  'Внeзапная смерть'],
 ['0114319',
  'Sydney Pollack',
  '$58,000,000 (estimated)',
  '$53,696,959',
  '2 hours 7 minutes',
  'Сабрина'],
 ['0113497',
  'Joe Johnston',
  '$65,000,000 (estimated)',
  '$262,821,940',
  '1 hour 44 minutes',
  'Джуманджи'],
 ['0113277',
  'Michael Mann',
  '$60,000,000 (estimated)',
  '$187,436,818',
  '2 hours 50 minutes',
  'Схватка'],
 ['0113228',
  'Howard Deutch',
  '$25,000,000 (estimated)',
  '$71,518,503',
  '1 hour 41 minutes',
  'Старые ворчуны разбушевались'],
 ['0113189',
  'Martin Campbell',
  '$60,000,000 (estimated)',
  '$352,194,034',
  '2 hours 10 minutes',
  'Золотой глаз'],
 ['0113041',
  'Ch

9.97 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 4.2. top_directors(self, n)

А если мы хотим узнать кто из директоров больше участвовал в большем числе фильмов, можно использовать этот метод.

Метод возвращает словарь, где директору соответствует количество фильмов, в которых он участвовал.

In [38]:
%%timeit -r 1 -n 1
top_dir = links.top_directors(10)
display(top_dir)

{'Forest Whitaker': 1,
 'John Lasseter': 1,
 'Peter Hyams': 1,
 'Sydney Pollack': 1,
 'Joe Johnston': 1,
 'Michael Mann': 1,
 'Howard Deutch': 1,
 'Martin Campbell': 1,
 'Charles Shyer': 1,
 'Peter Hewitt': 1}

6.28 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 4.3. most_expensive(self, n)

А какой фильм был самым дорогим? Сколько вложили средств в его производство?

Метод возвращает словарь, где названиям фильма соответствует его бюджет.

In [39]:
%%timeit -n1 -r1
display(links.most_expensive(10))

{'Джуманджи': 65000000.0,
 'Схватка': 60000000.0,
 'Золотой глаз': 60000000.0,
 'Сабрина': 58000000.0,
 'Внeзапная смерть': 35000000.0,
 'История игрушек': 30000000.0,
 'Отец невесты 2': 30000000.0,
 'Старые ворчуны разбушевались': 25000000.0,
 'В ожидании выдоха': 16000000.0,
 'Приключения Тома Сойера': 0.0}

1.45 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 4.4. most_profitable(self, n)

Самые дорогие фильмы это интересно, но хочется знать, кто сколько заработал.

Метод возвращает словарь, где названиям фильма соответсвует сумма заработанных денег.

In [40]:
%%timeit -n1 -r1
display(links.most_profitable(10))

{'История игрушек': 364436586.0,
 'Золотой глаз': 292194034.0,
 'Джуманджи': 197821940.0,
 'Схватка': 127436818.0,
 'В ожидании выдоха': 65452156.0,
 'Отец невесты 2': 46594107.0,
 'Старые ворчуны разбушевались': 46518503.0,
 'Внeзапная смерть': 29350171.0,
 'Приключения Тома Сойера': 23920048.0,
 'Сабрина': -4303041.0}

1.25 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 4.5. longest(self, n)

Если нужно скоротать время, хочется выяснить какой фильм самый продолжительный, чтобы было чем себя занят.

Метод возвращает словарь, где названию фильма соответсвует его продолжительность в минутах.

In [41]:
%%timeit -n1 -r1
display(links.longest(10))

{'Схватка': 170,
 'Золотой глаз': 130,
 'Сабрина': 127,
 'В ожидании выдоха': 124,
 'Внeзапная смерть': 111,
 'Отец невесты 2': 106,
 'Джуманджи': 104,
 'Старые ворчуны разбушевались': 101,
 'Приключения Тома Сойера': 97,
 'История игрушек': 81}

1.85 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 4.6* top_cost_per_minute(self, n)

Какой же из этих фильмов самый дорогой по минуте показа.

Метод top_cost_per_minute(self, n) предназначен для того, чтобы вернуть топ n фильмов с наивысшей стоимостью за минуту.
То есть, если у вас есть данные о стоимости фильма и его длительности, этот метод поможет найти те фильмы, которые имеют максимальное соотношение цена/время — самые дорогие по минуте показа.


In [42]:
%%timeit -n1 -r1
display(links.top_cost_per_minute(10))

{'Джуманджи': 625000.0,
 'Золотой глаз': 461538.46153846156,
 'Сабрина': 456692.91338582675,
 'История игрушек': 370370.3703703704,
 'Схватка': 352941.17647058825,
 'Внeзапная смерть': 315315.3153153153,
 'Отец невесты 2': 283018.8679245283,
 'Старые ворчуны разбушевались': 247524.75247524751,
 'В ожидании выдоха': 129032.25806451614,
 'Приключения Тома Сойера': 0.0}

1.49 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 4.7* get_imdb_rating(self, list_of_movies)

Интересно увидет рейтинги фильмов. 

Метод позволяет спарсить информацию и заполняет словарь id_movies, название фильма и рейтинг.

In [43]:
%%timeit -n1 -r1
links.get_imdb_rating(imdb_ids)
display(links.get_imdb_rating(imdb_ids))

[['0114709', 'История игрушек', '8.3/10'],
 ['0113277', 'Схватка', '8.3/10'],
 ['0113189', 'Золотой глаз', '7.2/10'],
 ['0113497', 'Джуманджи', '7.1/10'],
 ['0113228', 'Старые ворчуны разбушевались', '6.7/10'],
 ['0114319', 'Сабрина', '6.3/10'],
 ['0113041', 'Отец невесты 2', '6.1/10'],
 ['0114885', 'В ожидании выдоха', '6.0/10'],
 ['0114576', 'Внeзапная смерть', '5.9/10'],
 ['0112302', 'Приключения Тома Сойера', '5.5/10']]

23.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 5. Класс Tests

Про него говорили не писать, но он выполняет важную функцию- покрывает тестами все методы основных четырёх классов. Поэтому не можем его не упомянуть.

## Выводы
Мы увидели, что в начале XX века преобладали развлекательные и фантастические жанры, в 30-40-х — классическое и социально значимое кино. В 60-90-х вырос интерес к военным и научно-фантастическим фильмам, а после 2000-х появились более визуально насыщенные и интеллектуальные ленты с акцентом на психологию и атмосферу. Средний рейтинг стабилен, но внимание зрителей и разнообразие тем заметно выросли.